In [167]:
import pandas as pd

In [168]:
sheets = pd.read_excel("4BeatsQ1.xlsx", sheet_name=None)  # Read all sheets

In [169]:
from datetime import datetime

In [170]:
sheet = datetime.now().strftime('%A')  # Returns full day name (e.g. 'Friday') 

In [241]:
# sheet = 'Sunday'
# sheet ='Monday'
# sheet ='Tuesday'
# sheet ='Wednesday'
# sheet ='Thursday'
# sheet ='Friday'
# sheet ='Saturday'

In [171]:
sheet

'Friday'

In [242]:
df = sheets[sheet]

In [243]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,Longest Option,Shortest\nOption
1,NaN,Keyword1,Dhaka,NaN,NaN
2,NaN,Keyword2,Saturday,NaN,NaN
3,NaN,Keyword3,Baby,NaN,NaN
4,NaN,Keyword4,School,NaN,NaN
5,NaN,Keyword5,Cricket,NaN,NaN
6,NaN,Keyword6,Momey,NaN,NaN
7,NaN,Keyword7,Int,NaN,NaN
8,NaN,Keyword8,Look,NaN,NaN
9,NaN,Keyword9,Hello,NaN,NaN


In [244]:
df = df.drop(columns=['Unnamed: 0'])  # Drop column where all elements are NaN
df.columns = ['Keyword', 'Value', 'Longest Option', 'Shortest Option']  # Label columns for better readability

In [245]:
# Check if "Keyword" is present in the first row (For sheets like Tuesday)
if not any(cell.startswith('Keyword') for cell in df.iloc[0].astype(str).values):
    df = df.drop(df.index[0])  # Remove row with partial/old labels

In [246]:
# Reset index of the DataFrame to ensure the indices align
df.reset_index(drop=True, inplace=True)

In [247]:
df

,Keyword,Value,Longest Option,Shortest Option
0,Keyword1,Dhaka,NaN,NaN
1,Keyword2,Saturday,NaN,NaN
2,Keyword3,Baby,NaN,NaN
3,Keyword4,School,NaN,NaN
4,Keyword5,Cricket,NaN,NaN
5,Keyword6,Momey,NaN,NaN
6,Keyword7,Int,NaN,NaN
7,Keyword8,Look,NaN,NaN
8,Keyword9,Hello,NaN,NaN
9,Keyword10,By,NaN,NaN


In [248]:
keywords = df['Value'].dropna().tolist()

In [249]:
keywords

['Dhaka',
 'Saturday',
 'Baby',
 'School',
 'Cricket',
 'Momey',
 'Int',
 'Look',
 'Hello',
 'By']

In [250]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [251]:
import time

In [252]:
def get_auto_suggestions(browser):
    elements = browser.find_elements(By.XPATH, '//*[@id="Alh6id"]/div[1]/div/ul/li')
    suggestions = [elem.text for elem in elements if elem.text]
    return suggestions

**Program assumes Firefox is installed and geckodriver is in the same directory**

In [253]:
browser = webdriver.Firefox() 
browser.maximize_window()  
browser.get('http://www.google.com')
assert 'Google' in browser.title

for index, keyword in enumerate(keywords):
    elem = browser.find_element(By.NAME, 'q')  
    elem.clear()  # Clear search box of any previous search keywords
    elem.send_keys(keyword)  
    
    time.sleep(3)  # Allow autocomplete suggestions to load

    suggestions_list = get_auto_suggestions(browser)
    
    if suggestions_list:
        longest_option = max(suggestions_list, key=len)
        shortest_option = min(suggestions_list, key=len)
    else:
        longest_option = shortest_option = ''
    
    df.at[index, 'Longest Option'] = longest_option
    df.at[index, 'Shortest Option'] = shortest_option
    
    time.sleep(3)  # Wait a few seconds before the next search

browser.close()

In [254]:
df

,Keyword,Value,Longest Option,Shortest Option
0,Keyword1,Dhaka,ঢাকা শিক্ষা বোর্ড\nমাধ্যমিক ও উচ্চমাধ্যমিক শিক...,ঢাকা পোস্ট
1,Keyword2,Saturday,Saturday Night Live\nস্যাটারডে নাইট লাইভ — টিভ...,saturday
2,Keyword3,Baby,BABY DRIVER\nবেবি ড্রাইভার — ২০১৭ সালের চলচ্চিত্র,baby pic
3,Keyword4,School,School 2017\nটিভি প্রোগ্রাম,school
4,Keyword5,Cricket,cricket world cup\nআইসিসি পুরুষ টি২০ বিশ্বকাপ ...,cricket
5,Keyword6,Momey,money cannot bring happiness story,money
6,Keyword7,Int,Inter Miami\nইন্টার মায়ামি সিএফ — ফুটবল টীম,ইন্টারনেট
7,Keyword8,Look,lookism\nলুকিজম — কমিক বই ক্রম,look
8,Keyword9,Hello,Hello Hello! (Featuring the Super Simple Puppe...,hello kitty
9,Keyword10,By,ব্যক্তিত্ব নিয়ে উক্তি,ব্যাংক


In [255]:
import openpyxl

In [256]:
# Update workbook with modified sheet
sheets[sheet] = df

with pd.ExcelWriter("4BeatsQ1.xlsx", engine='openpyxl', mode='w') as f:
    for sheet, content in sheets.items():
        content.to_excel(f, sheet_name=sheet, index=False)